<a href="https://colab.research.google.com/github/ebrevdo/firearms_causes_of_death/blob/main/child_death_by_state_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>